<center><h1>Real world application: linear regression in logrithm forms</h1></center>

---

## Check the following before running the code

### (a) Read "README.pdf" in this folder first, which introduces the package

### (b) Before replication, delete all .p files in the "./numerical_result" folder. The .p files record the numerical results of the our computation.

### (c) To avoid confusion, reset your kernel before you running the notebook (to clear memory): 
* <font size="4.5"> In Jupyter Notebook/Lab : go to Menu "Kernel" $\rightarrow$ "Restart Kernel and clear all outputs". </font> 

### (d) To evaluate the code for simulation replication in Jupyter Notebook/Lab,
* <font size="4.5"> click : Menu "Kernel" $\rightarrow$ "Restart Kernel and Run All Cells" </font>
* <font size="4.5"> or, select a cell of code, press "shift" and "enter". Run all cells to avoid errors </font>

### (e) Check "joblib", "scikit-learn", "numpy", "matplotlib" and "tqdm" are installed. If not,
* <font size="4.5"> we highly recommend installing Anaconda3 version 2020-11 directly to avoid package management (all packages mentioned above are installed by default).</font>

---

# #1. reset python kernel

In [1]:
%reset -f

---

# #2. import packages

In [2]:
import numpy             as np
import statsmodels.api   as sm
import pandas            as pd
import matplotlib.pyplot as plt
import statsmodels.api   as sm

from sklearn.linear_model       import LassoLarsCV, LassoCV, ElasticNetCV
from solar                      import solar
from statsmodels.iolib.summary2 import summary_col
from collections                import OrderedDict

## make sure we use the Intel MKL C++/Fortran compiler for maximum performance.

In [3]:
import mkl

mkl.get_version_string()

'Intel(R) oneAPI Math Kernel Library Version 2021.4-Product Build 20210904 for Intel(R) 64 architecture applications'

In [4]:
print('This was obtained using the following Numpy configuration:')

np.show_config()

This was obtained using the following Numpy configuration:
blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/ning/anaconda3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/ning/anaconda3/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/ning/anaconda3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/ning/anaconda3/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/ning/anaconda3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/ning/anaconda3/include']
lapack_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/ning/anaconda3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/ning/anaconda3/include']
Supported SIMD extensions in this NumPy install:
    

---

# #3. load house price data

In [5]:
linTB = pd.read_csv("Data2010.csv")

## #3(a). rename variables

In [6]:
linTB.rename(columns={'Tot_P_P':'TotPop',
                      'Mean_age_persons':'AgeSA1',
                      'Australian_citizen_P':'Aus',
                      'Average_num_psns_per_bedroom':'psn-per-bedroom',
                      'Average_household_size':'household_size',
                      'FinalResultEventPrice': 'Price',
                      'Lang_spoken_home_Eng_only_P':'Eng-speak',
                      'Mean_mortgage_repay_monthly':'Mortgage',
                      'Mean_Tot_prsnl_inc_weekly':'Inc',
                      'Mean_rent_weekly':'Rent',
                      'Mean_Tot_fam_inc_weekly':'FamInc'}, 
                 inplace=True)
linTB = linTB.drop(['Unnamed: 0'],axis=1)
linTB = linTB.dropna()
linTB['const'] = 1
linTB.columns


Index(['TotPop', 'Eng-speak', 'Aus', 'AgeSA1', 'Mortgage', 'Inc', 'Rent',
       'FamInc', 'psn-per-bedroom', 'household_size', 'TVO2010', 'TPO2010',
       'TVO2009', 'TPO2009', 'Suburb_Area', 'AreaSize', 'Bedrooms', 'Baths',
       'Parking', 'Airport', 'Beach', 'Cemetery', 'ChildCare',
       'CommunityFacility', 'Club', 'Gaol', 'GeneralHospital', 'GolfCourt',
       'High', 'Lib', 'MedCenter', 'Museum', 'Park', 'PO', 'Police',
       'PreSchool', 'PrimaryHigh', 'Primary', 'RailStat', 'Rubbish', 'Sewage',
       'SportsCenter', 'SportsCourtField', 'Swimming', 'Tertiary', 'DistBound',
       'ICSEA', 'ReadingY3', 'WritingY3', 'SpellingY3', 'GrammarY3',
       'NumeracyY3', 'ReadingY5', 'WritingY5', 'SpellingY5', 'GrammarY5',
       'NumeracyY5', 'Price', 'const'],
      dtype='object')

In [7]:
linTB.to_csv('House2010_linear.csv')
Y_linear = linTB[['Price']]
X_linear = linTB.drop(columns=['Price'])
X_linear.columns

Index(['TotPop', 'Eng-speak', 'Aus', 'AgeSA1', 'Mortgage', 'Inc', 'Rent',
       'FamInc', 'psn-per-bedroom', 'household_size', 'TVO2010', 'TPO2010',
       'TVO2009', 'TPO2009', 'Suburb_Area', 'AreaSize', 'Bedrooms', 'Baths',
       'Parking', 'Airport', 'Beach', 'Cemetery', 'ChildCare',
       'CommunityFacility', 'Club', 'Gaol', 'GeneralHospital', 'GolfCourt',
       'High', 'Lib', 'MedCenter', 'Museum', 'Park', 'PO', 'Police',
       'PreSchool', 'PrimaryHigh', 'Primary', 'RailStat', 'Rubbish', 'Sewage',
       'SportsCenter', 'SportsCourtField', 'Swimming', 'Tertiary', 'DistBound',
       'ICSEA', 'ReadingY3', 'WritingY3', 'SpellingY3', 'GrammarY3',
       'NumeracyY3', 'ReadingY5', 'WritingY5', 'SpellingY5', 'GrammarY5',
       'NumeracyY5', 'const'],
      dtype='object')

## #3(c). drop 0

### return the observations with zero in those variables

In [8]:
logTB = linTB
logTB = logTB[(logTB != 0).all(1)]

## #3(d). generate log variables

In [9]:
logTB[['logPrice']]     = np.log(logTB[['Price']])
logTB[['logMortgage']]  = np.log(logTB[['Mortgage']])
logTB[['logRent']]      = np.log(logTB[['Rent']])
logTB[['logFamInc']]    = np.log(logTB[['FamInc']])
logTB[['logPersonInc']] = np.log(logTB[['Inc']])

logTB = logTB.drop(['Price','Mortgage', 'Rent', 'FamInc', 'Inc'], axis=1)

/var/folders/wq/pf376vkd6m55xggd84pw1t180000gn/T/ipykernel_22120/765982306.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logTB[['logPrice']]     = np.log(logTB[['Price']])
/var/folders/wq/pf376vkd6m55xggd84pw1t180000gn/T/ipykernel_22120/765982306.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logTB[['logMortgage']]  = np.log(logTB[['Mortgage']])
/var/folders/wq/pf376vkd6m55xggd84pw1t180000gn/T/ipykernel_22120/765982306.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [10]:
logTB.to_csv('House2010_log.csv')
X_log = logTB.drop(columns=['logPrice'])
Y_log = logTB[['logPrice']]
X_log.columns

Index(['TotPop', 'Eng-speak', 'Aus', 'AgeSA1', 'psn-per-bedroom',
       'household_size', 'TVO2010', 'TPO2010', 'TVO2009', 'TPO2009',
       'Suburb_Area', 'AreaSize', 'Bedrooms', 'Baths', 'Parking', 'Airport',
       'Beach', 'Cemetery', 'ChildCare', 'CommunityFacility', 'Club', 'Gaol',
       'GeneralHospital', 'GolfCourt', 'High', 'Lib', 'MedCenter', 'Museum',
       'Park', 'PO', 'Police', 'PreSchool', 'PrimaryHigh', 'Primary',
       'RailStat', 'Rubbish', 'Sewage', 'SportsCenter', 'SportsCourtField',
       'Swimming', 'Tertiary', 'DistBound', 'ICSEA', 'ReadingY3', 'WritingY3',
       'SpellingY3', 'GrammarY3', 'NumeracyY3', 'ReadingY5', 'WritingY5',
       'SpellingY5', 'GrammarY5', 'NumeracyY5', 'const', 'logMortgage',
       'logRent', 'logFamInc', 'logPersonInc'],
      dtype='object')

## check correlation among all variables

In [11]:
X_log.drop(columns=['const']).corr()

,TotPop,Eng-speak,Aus,AgeSA1,psn-per-bedroom,household_size,TVO2010,TPO2010,TVO2009,TPO2009,...,NumeracyY3,ReadingY5,WritingY5,SpellingY5,GrammarY5,NumeracyY5,logMortgage,logRent,logFamInc,logPersonInc
TotPop,1.000000,0.446759,0.811870,-0.226880,0.293217,-0.100453,0.073287,0.094085,0.069827,0.087163,...,-0.053081,-0.114005,-0.083455,-0.038862,-0.097933,-0.044633,-0.056936,0.042300,-0.063915,0.008896
Eng-speak,0.446759,1.000000,0.685884,0.136034,-0.425182,-0.181800,-0.141391,-0.168526,-0.117459,-0.155269,...,0.120956,0.299560,0.115671,-0.019657,0.260517,0.071445,0.288768,0.176285,0.543959,0.517215
Aus,0.811870,0.685884,1.000000,0.094539,-0.141462,0.103852,-0.183825,-0.183372,-0.179502,-0.189033,...,-0.053529,-0.036554,-0.064526,-0.052185,-0.038443,-0.051935,0.036190,-0.014366,0.065877,0.037716
AgeSA1,-0.226880,0.136034,0.094539,1.000000,-0.557938,0.169395,-0.261133,-0.257870,-0.260042,-0.252402,...,0.024359,0.074088,0.023159,-0.000914,0.070340,0.007742,0.172088,-0.057359,0.120974,-0.092774
psn-per-bedroom,0.293217,-0.425182,-0.141462,-0.557938,1.000000,-0.215892,0.339445,0.340161,0.322691,0.327790,...,-0.096159,-0.258094,-0.128285,-0.013226,-0.234769,-0.063565,-0.352967,-0.090807,-0.460331,-0.228175
household_size,-0.100453,-0.181800,0.103852,0.169395,-0.215892,1.000000,-0.269701,-0.226392,-0.280639,-0.234754,...,-0.104145,-0.174856,-0.083672,-0.033719,-0.166272,-0.065168,0.049089,0.019097,-0.152279,-0.484238
TVO2010,0.073287,-0.141391,-0.183825,-0.261133,0.339445,-0.269701,1.000000,0.941039,0.981773,0.937818,...,-0.028115,-0.056718,0.009739,0.066351,-0.032904,0.014634,-0.115994,0.022334,-0.074767,0.015133
TPO2010,0.094085,-0.168526,-0.183372,-0.257870,0.340161,-0.226392,0.941039,1.000000,0.918736,0.992204,...,0.029600,-0.017522,0.041159,0.096467,0.003102,0.076676,-0.064870,0.074552,-0.042652,0.017141
TVO2009,0.069827,-0.117459,-0.179502,-0.260042,0.322691,-0.280639,0.981773,0.918736,1.000000,0.915290,...,-0.031647,-0.047698,0.024838,0.071765,-0.018475,0.013650,-0.087019,0.044776,-0.035943,0.052502
TPO2009,0.087163,-0.155269,-0.189033,-0.252402,0.327790,-0.234754,0.937818,0.992204,0.915290,1.000000,...,0.033981,-0.013305,0.044001,0.092469,0.006399,0.069867,-0.051810,0.081555,-0.021384,0.033847


---

# #4. generate X and Y for variable selection

In [12]:
X_linear_input = np.array(X_linear)
Y_linear_input = np.array(Y_linear)
X_log_input    = np.array(X_log)
Y_log_input    = np.array(Y_log)

In [13]:
obs     = X_linear_input.shape[0]
obs_log = X_log_input.shape[0]

Y_linear_input.shape = (obs,)
Y_log_input.shape    = (obs_log,)

---

# #5. compute CV coordinate descend (CV-cd), CV lars for lasso (CV-lars-lasso), CV elastic net (CV-en) and solar regression 

In [14]:
# CV coordinate descend (CV-cd)

RegLog1 = LassoCV(cv=10, normalize = True, random_state=0)
RegLog1.fit(X_log_input, Y_log_input)

LogLassoCV_ceof = RegLog1.coef_
LogLassoCV_ind  = np.nonzero(LogLassoCV_ceof)


# CV-lars-lasso

np.random.seed(0)

RegLog2 = LassoLarsCV(cv=10)
RegLog2.fit(X_log_input,Y_log_input)

LogLasso_lars_ceof = RegLog2.coef_
LogLasso_lars_ind  = np.nonzero(LogLasso_lars_ceof)

# CV-en

RegLog3 = ElasticNetCV(l1_ratio=[.5, .7, .9, .95],normalize=True, random_state=0)
RegLog3.fit(X_log_input,Y_log_input)

# solar

Y_log_input.shape = (obs_log,1)

np.random.seed(0)

RegLog4 = solar( X_log_input,Y_log_input, 10, -0.02, lasso = False)
LogSolar_coef, LogOpt_c, LogTest_error, LogQc_list, _, _, _, _, _ = RegLog4.fit()

LogSolar_ind = np.nonzero(LogSolar_coef)[0]

---

# #8. the results of variable selection

## #6(c). variable selection by lasso, solar and elastic net

### variables selected by CV-lars-lasso

In [15]:
LogLasso_lars_active = X_log.columns[LogLasso_lars_ind]
print(np.count_nonzero(LogLasso_lars_ind))
print(LogLasso_lars_active)

35
Index(['TotPop', 'AgeSA1', 'psn-per-bedroom', 'household_size', 'TPO2010',
       'Suburb_Area', 'AreaSize', 'Bedrooms', 'Baths', 'Parking', 'Airport',
       'Beach', 'ChildCare', 'Club', 'GeneralHospital', 'GolfCourt', 'High',
       'MedCenter', 'Museum', 'PO', 'Police', 'PreSchool', 'PrimaryHigh',
       'Primary', 'SportsCenter', 'SportsCourtField', 'Swimming', 'Tertiary',
       'DistBound', 'ICSEA', 'ReadingY3', 'WritingY3', 'NumeracyY3',
       'logMortgage', 'logRent', 'logFamInc'],
      dtype='object')


### variables selected by CV-cd

In [16]:
LogLassoCV_active = X_log.columns[LogLassoCV_ind]
print(np.count_nonzero(LogLassoCV_ceof))
print(LogLassoCV_active)

36
Index(['TotPop', 'AgeSA1', 'psn-per-bedroom', 'household_size', 'TPO2010',
       'Suburb_Area', 'AreaSize', 'Bedrooms', 'Baths', 'Parking', 'Airport',
       'Beach', 'ChildCare', 'Club', 'GeneralHospital', 'GolfCourt', 'High',
       'MedCenter', 'Museum', 'PO', 'Police', 'PreSchool', 'PrimaryHigh',
       'Primary', 'SportsCenter', 'SportsCourtField', 'Swimming', 'Tertiary',
       'DistBound', 'ICSEA', 'ReadingY3', 'WritingY3', 'NumeracyY3',
       'logMortgage', 'logRent', 'logFamInc'],
      dtype='object')


### variables selected by solar

In [17]:
LogSolar_active = X_log.columns[LogSolar_ind]
print(np.count_nonzero(LogSolar_coef))
print(LogSolar_active)

11
Index(['AgeSA1', 'Bedrooms', 'Baths', 'Parking', 'Beach', 'ChildCare', 'Gaol',
       'ICSEA', 'logMortgage', 'logRent', 'logFamInc'],
      dtype='object')


### variables selected by CV-en

In [18]:
LogEn_coef   = RegLog3.coef_
LogEn_ind    = np.nonzero(LogEn_coef)
LogEn_active = X_log.columns[LogEn_ind]
print("number of variables selected by CV-EN: ", np.count_nonzero(LogEn_coef))
print("variablesselected by CV-EN: ", LogEn_active)

number of variables selected by CV-EN:  45
variablesselected by CV-EN:  Index(['TotPop', 'AgeSA1', 'psn-per-bedroom', 'household_size', 'TPO2010',
       'TVO2009', 'TPO2009', 'Suburb_Area', 'AreaSize', 'Bedrooms', 'Baths',
       'Parking', 'Airport', 'Beach', 'ChildCare', 'CommunityFacility', 'Club',
       'Gaol', 'GeneralHospital', 'GolfCourt', 'High', 'MedCenter', 'Museum',
       'Park', 'PO', 'Police', 'PreSchool', 'PrimaryHigh', 'Primary',
       'Rubbish', 'SportsCenter', 'Swimming', 'Tertiary', 'DistBound', 'ICSEA',
       'ReadingY3', 'WritingY3', 'SpellingY3', 'NumeracyY3', 'WritingY5',
       'SpellingY5', 'GrammarY5', 'logMortgage', 'logRent', 'logFamInc'],
      dtype='object')


In [19]:
LogEn_inactive = np.setdiff1d(np.arange(0,58), np.array(LogEn_ind[0]) )
print("number of variable purged by CV-EN: ", LogEn_inactive.shape[0])

print("variable purged by CV-EN: ", X_log.columns[LogEn_inactive])

number of variable purged by CV-EN:  13
variable purged by CV-EN:  Index(['Eng-speak', 'Aus', 'TVO2010', 'Cemetery', 'Lib', 'RailStat', 'Sewage',
       'SportsCourtField', 'GrammarY3', 'ReadingY5', 'NumeracyY5', 'const',
       'logPersonInc'],
      dtype='object')


# #9. Post-selection regression

## #7(a). post-elastic-net OLS

In [20]:
X_LogEn   = X_log[LogEn_active]
X_LogEn   = sm.add_constant(X_LogEn)
OLS_LogEn = sm.OLS(Y_log, X_LogEn)

OLS_LogEn_result = OLS_LogEn.fit()
print(OLS_LogEn_result.summary())

                            OLS Regression Results                            
Dep. Variable:               logPrice   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     852.6
Date:                Sat, 04 Jun 2022   Prob (F-statistic):               0.00
Time:                        04:10:16   Log-Likelihood:                -679.32
No. Observations:               11796   AIC:                             1451.
Df Residuals:                   11750   BIC:                             1790.
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 8.8897      0.16

## #7(b). post-lasso (CV-cd) OLS

In [21]:
X_LogLassoCV   = X_log[LogLassoCV_active]
Y_LogLassoCV   = Y_log
X_LogLassoCV   = sm.add_constant(X_LogLassoCV)
OLS_LogLassoCV = sm.OLS(Y_LogLassoCV, X_LogLassoCV)

OLS_LogLassoCV_result = OLS_LogLassoCV.fit()
print(OLS_LogLassoCV_result.summary())

                            OLS Regression Results                            
Dep. Variable:               logPrice   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     1062.
Date:                Sat, 04 Jun 2022   Prob (F-statistic):               0.00
Time:                        04:10:16   Log-Likelihood:                -700.48
No. Observations:               11796   AIC:                             1475.
Df Residuals:                   11759   BIC:                             1748.
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                8.7614      0.151  

## #7(c). post-lasso (lars-lasso) OLS

In [22]:
X_LogLars   = X_log[LogLasso_lars_active]
Y_LogLars   = Y_log
X_LogLars   = sm.add_constant(X_LogLars)
OLS_LogLars = sm.OLS(Y_LogLars, X_LogLars)

OLS_LogLars_result = OLS_LogLars.fit()
print(OLS_LogLars_result.summary())

                            OLS Regression Results                            
Dep. Variable:               logPrice   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     1062.
Date:                Sat, 04 Jun 2022   Prob (F-statistic):               0.00
Time:                        04:10:16   Log-Likelihood:                -700.48
No. Observations:               11796   AIC:                             1475.
Df Residuals:                   11759   BIC:                             1748.
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                8.7614      0.151  

## #7(d). post-solar OLS

In [23]:
X_Gaol     = X_log[LogSolar_active]
X_Gaol     = sm.add_constant(X_Gaol)
OLS_LogSolar  = sm.OLS(Y_log, X_Gaol)
OLS_LogSolar_result = OLS_LogSolar.fit()
print(OLS_LogSolar_result.summary())

                            OLS Regression Results                            
Dep. Variable:               logPrice   R-squared:                       0.730
Model:                            OLS   Adj. R-squared:                  0.730
Method:                 Least Squares   F-statistic:                     2900.
Date:                Sat, 04 Jun 2022   Prob (F-statistic):               0.00
Time:                        04:10:16   Log-Likelihood:                -1507.3
No. Observations:               11796   AIC:                             3039.
Df Residuals:                   11784   BIC:                             3127.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           7.2109      0.105     68.413      

## hold-out average result

In [24]:
half_size = int(np.round(X_Gaol.shape[0]/2))

X_half_1 = X_Gaol[0:half_size]
Y_half_1 = Y_log[0:half_size]
X_half_2 = X_Gaol[half_size:]
Y_half_2 = Y_log[half_size:]

active_var = [str(i) for i in LogSolar_active]
XX_half_1 = X_half_1[LogSolar_active]
YY_half_1 = Y_half_1
XX_half_1 = sm.add_constant(XX_half_1)
reg_half_1 = sm.OLS(YY_half_1, XX_half_1)

result_half_1 = reg_half_1.fit()

XX_half_2 = X_half_2[LogSolar_active]
YY_half_2 = Y_half_2
XX_half_2 = sm.add_constant(XX_half_2)
reg_half_2 = sm.OLS(YY_half_2, XX_half_2)

result_half_2 = reg_half_2.fit()

In [25]:
#hold-out average R^2
average_R2    = (result_half_1.rsquared + result_half_2.rsquared)/2

#hold-out average t-value
AgeSA1_t      = (result_half_1.tvalues[1]  + result_half_2.tvalues[1] )/2
Bedrooms_t    = (result_half_1.tvalues[2]  + result_half_2.tvalues[2] )/2
Baths_t       = (result_half_1.tvalues[3]  + result_half_2.tvalues[3] )/2
Parking_t     = (result_half_1.tvalues[4]  + result_half_2.tvalues[4] )/2
Beach_t       = (result_half_1.tvalues[5]  + result_half_2.tvalues[5] )/2
ChildCare_t   = (result_half_1.tvalues[6]  + result_half_2.tvalues[6] )/2
Gaol_t        = (result_half_1.tvalues[7]  + result_half_2.tvalues[7] )/2
ICSEA_t       = (result_half_1.tvalues[8]  + result_half_2.tvalues[8] )/2
logMortgage_t = (result_half_1.tvalues[9]  + result_half_2.tvalues[9] )/2
logRent_t     = (result_half_1.tvalues[10] + result_half_2.tvalues[10])/2
logFamInc_t   = (result_half_1.tvalues[11] + result_half_2.tvalues[11])/2

#hold-out average p-value
AgeSA1_p      = (result_half_1.pvalues[1]  + result_half_2.pvalues[1] )/2
Bedrooms_p    = (result_half_1.pvalues[2]  + result_half_2.pvalues[2] )/2
Baths_p       = (result_half_1.pvalues[3]  + result_half_2.pvalues[3] )/2
Parking_p     = (result_half_1.pvalues[4]  + result_half_2.pvalues[4] )/2
Beach_p       = (result_half_1.pvalues[5]  + result_half_2.pvalues[5] )/2
ChildCare_p   = (result_half_1.pvalues[6]  + result_half_2.pvalues[6] )/2
Gaol_p        = (result_half_1.pvalues[7]  + result_half_2.pvalues[7] )/2
ICSEA_p       = (result_half_1.pvalues[8]  + result_half_2.pvalues[8] )/2
logMortgage_p = (result_half_1.pvalues[9]  + result_half_2.pvalues[9] )/2
logRent_p     = (result_half_1.pvalues[10] + result_half_2.pvalues[10])/2
logFamInc_p   = (result_half_1.pvalues[11] + result_half_2.pvalues[11])/2

#hold-out average se
AgeSA1_se      = (result_half_1.bse[1]  + result_half_2.bse[1] )/2
Bedrooms_se    = (result_half_1.bse[2]  + result_half_2.bse[2] )/2
Baths_se       = (result_half_1.bse[3]  + result_half_2.bse[3] )/2
Parking_se     = (result_half_1.bse[4]  + result_half_2.bse[4] )/2
Beach_se       = (result_half_1.bse[5]  + result_half_2.bse[5] )/2
ChildCare_se   = (result_half_1.bse[6]  + result_half_2.bse[6] )/2
Gaol_se        = (result_half_1.bse[7]  + result_half_2.bse[7] )/2
ICSEA_se       = (result_half_1.bse[8]  + result_half_2.bse[8] )/2
logMortgage_se = (result_half_1.bse[9]  + result_half_2.bse[9] )/2
logRent_se     = (result_half_1.bse[10] + result_half_2.bse[10])/2
logFamInc_se   = (result_half_1.bse[11] + result_half_2.bse[11])/2

df_hor = pd.concat([pd.DataFrame({'variable name':active_var}), 
                    pd.DataFrame({'se':[AgeSA1_se, Bedrooms_se, Baths_se, Parking_se, Beach_se,
                                        ChildCare_se, Gaol_se, ICSEA_se, logMortgage_se, logRent_se, 
                                        logFamInc_se,],
                                  't-val':[AgeSA1_t, Bedrooms_t, Baths_t, Parking_t, Beach_t,
                                           ChildCare_t, Gaol_t, ICSEA_t, logMortgage_t, logRent_t,
                                           logFamInc_t], 
                                  'p-val': [AgeSA1_p, Bedrooms_p, Baths_p, Parking_p, Beach_p,
                                            ChildCare_p, Gaol_p, ICSEA_p, logMortgage_p, logRent_p,
                                            logFamInc_p]})], 
                   axis=1, join='inner')

df_hor.round(3)

,variable name,se,t-val,p-val
0,AgeSA1,0.001,18.129,0.000
1,Bedrooms,0.005,47.236,0.000
2,Baths,0.006,13.347,0.000
3,Parking,0.005,15.371,0.000
4,Beach,0.201,-11.379,0.000
5,ChildCare,0.157,-15.217,0.000
6,Gaol,0.203,0.617,0.001
7,ICSEA,0.000,9.134,0.000
8,logMortgage,0.017,14.862,0.000
9,logRent,0.012,6.002,0.000


In [26]:
print("the hold-out average R2 is ", average_R2.round(2))

the hold-out average R2 is  0.73


## #7(e). generate a comparison table for post-selection OLS result (slope, significance, $R^2$)

In [27]:
info_dict={'R-squared' : lambda x: f"{x.rsquared:.2f}",
           'No. observations' : lambda x: f"{int(x.nobs):d}"}

results_table = summary_col(results=[OLS_LogEn_result, OLS_LogLassoCV_result, 
                                     OLS_LogLars_result, OLS_LogSolar_result],
                            float_format='%0.2f',
                            stars = True,
                            model_names=['post-en',
                                         'post-lasso(lars)',
                                         'post-larsso(cd)',
                                         'post-solar'],
                            info_dict=info_dict,
                            regressor_order=['const',
                                             'age',
                                             'Bedrooms',
                                             'Baths',
                                             'Parking',
                                             'Beach',
                                             'ChildCare',
                                             'Gaol',
                                             'ICSEA',
                                             'logMortage',
                                             'logRent',
                                             'logFamInc'])

results_table.add_title('Table 1 - Post-selection OLS comparison')

print(results_table)

               Table 1 - Post-selection OLS comparison
                  post-en  post-lasso(lars) post-larsso(cd) post-solar
----------------------------------------------------------------------
const             8.89***  8.76***          8.76***         7.21***   
                  (0.16)   (0.15)           (0.15)          (0.11)    
Bedrooms          0.21***  0.21***          0.21***         0.23***   
                  (0.00)   (0.00)           (0.00)          (0.00)    
Baths             0.09***  0.10***          0.10***         0.09***   
                  (0.00)   (0.00)           (0.00)          (0.00)    
Parking           0.08***  0.08***          0.08***         0.08***   
                  (0.00)   (0.00)           (0.00)          (0.00)    
Beach             -2.48*** -2.21***         -2.21***        -2.45***  
                  (0.27)   (0.11)           (0.11)          (0.14)    
ChildCare         -4.46*** -4.49***         -4.49***        -2.45***  
                  (0.1

# #8. rectified solar selection

## #8(a). generate correlation table of all variables selected by solar

In [28]:
X_log[LogSolar_active].corr()

,AgeSA1,Bedrooms,Baths,Parking,Beach,ChildCare,Gaol,ICSEA,logMortgage,logRent,logFamInc
AgeSA1,1.000000,0.310088,0.166457,0.217984,0.012365,0.180460,0.124008,0.017711,0.172088,-0.057359,0.120974
Bedrooms,0.310088,1.000000,0.639381,0.477543,0.032923,0.243481,0.180793,-0.043061,0.173557,0.023377,0.094320
Baths,0.166457,0.639381,1.000000,0.427226,-0.075538,0.051809,0.026854,0.098237,0.215537,0.156228,0.199941
Parking,0.217984,0.477543,0.427226,1.000000,-0.008247,0.149590,0.091177,-0.022136,0.128067,0.043028,0.091189
Beach,0.012365,0.032923,-0.075538,-0.008247,1.000000,0.041460,0.532753,-0.259973,-0.231921,-0.251981,-0.340208
ChildCare,0.180460,0.243481,0.051809,0.149590,0.041460,1.000000,0.750484,-0.396499,-0.210664,-0.227383,-0.236176
Gaol,0.124008,0.180793,0.026854,0.091177,0.532753,0.750484,1.000000,-0.234434,-0.187952,-0.196900,-0.173732
ICSEA,0.017711,-0.043061,0.098237,-0.022136,-0.259973,-0.396499,-0.234434,1.000000,0.346822,0.364504,0.568851
logMortgage,0.172088,0.173557,0.215537,0.128067,-0.231921,-0.210664,-0.187952,0.346822,1.000000,0.306933,0.551085
logRent,-0.057359,0.023377,0.156228,0.043028,-0.251981,-0.227383,-0.196900,0.364504,0.306933,1.000000,0.482644


## #8(b). compute $\mathrm{corr} \left( \cdot , \mathrm{goal} \right)$ of all variables

In [29]:
goal_corr_table = X_log.corr()['Gaol']

### focus on those with $\left\vert \mathrm{corr} \left( \cdot , \mathrm{goal} \right) \right\vert \geqslant 0.3$

In [30]:
goal_corr_table[np.abs(goal_corr_table)>=0.3]

household_size       0.304713
Airport              0.708315
Beach                0.532753
Cemetery             0.404004
ChildCare            0.750484
CommunityFacility    0.317501
Club                 0.397920
Gaol                 1.000000
GeneralHospital      0.464019
GolfCourt            0.468939
PreSchool            0.468674
Rubbish              0.662911
Sewage               0.453901
Swimming             0.375316
Tertiary             0.481035
Name: Gaol, dtype: float64

### focus on those with $\left\vert \mathrm{corr} \left( \cdot , \mathrm{goal} \right) \right\vert \geqslant 0.5$

In [31]:
goal_corr_table[np.abs(goal_corr_table)>=0.5]

Airport      0.708315
Beach        0.532753
ChildCare    0.750484
Gaol         1.000000
Rubbish      0.662911
Name: Gaol, dtype: float64

## #8(c). check irrepresentable condition 

## $\mathrm{Gaol} = X_{Goal}\gamma + e$, where $X_{Goal} = \left[ \mathrm{Airport}, \mathrm{ChildCare}, \mathrm{Rubbish}, \mathrm{Beach} \right]$

In [32]:
X_Gaol_group = X_log[['Airport', 'ChildCare', 'Rubbish', 'Beach']]
Y_Gaol_group = X_log[['Gaol']]

Y_Gaol_group = (Y_Gaol_group - Y_Gaol_group.mean()) / Y_Gaol_group.std()
X_Gaol_group = (X_Gaol_group - X_Gaol_group.mean()) / X_Gaol_group.std()

X_Gaol_group = sm.add_constant(X_Gaol_group)

OLS_Gaol = sm.OLS(Y_Gaol_group, X_Gaol_group)

OLS_Gaol_result = OLS_Gaol.fit()
print(OLS_Gaol_result.summary())

                            OLS Regression Results                            
Dep. Variable:                   Gaol   R-squared:                       0.883
Model:                            OLS   Adj. R-squared:                  0.883
Method:                 Least Squares   F-statistic:                 2.230e+04
Date:                Sat, 04 Jun 2022   Prob (F-statistic):               0.00
Time:                        04:10:16   Log-Likelihood:                -4069.7
No. Observations:               11796   AIC:                             8149.
Df Residuals:                   11791   BIC:                             8186.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -8.785e-15      0.003  -2.79e-12      1.0

### check if $\left\Vert \gamma \right\Vert_1 \geqslant 1$

In [33]:
print('the abs value of regression coef: ', np.sum(np.abs(OLS_Gaol_result.params)))

the abs value of regression coef:  1.3683947330011703


In [1]:
!rm -rf application_Houseprice_log.html
!jupyter nbconvert --to html application_Houseprice_log.ipynb 

[NbConvertApp] Converting notebook application_Houseprice_log.ipynb to html
[NbConvertApp] Writing 758423 bytes to application_Houseprice_log.html
